In [19]:
# Python Connector Example with Snowflake Python Connector, libs setup
# in a Jupyter Notebook.  Examples of One Hot Encoding as a Python Function.  Examples of Correlation as 
# Python based Functions.
#
#
# Feb 2022 - Ed Crean
#
# You have to install some libs for the snowflake connector to work.
#
#   Upgrade pip3: pip3 install --upgrade setuptools pip
#           python3 -m pip install --upgrade pip
#   Install: python3 -V. (Came back with 3.8.9)
#            pip3 install -r https://raw.githubusercontent.com/snowflakedb/snowflake-connector-python/v2.6.2/tested_requirements/requirements_38.reqs
#            -- This command will install the latest
#            pip3 install --upgrade snowflake-connector-python
#            -- This command will install specifically 2.6.2 version
#            pip3 install snowflake-connector-python=2.6.2
#
# IMPORTS
import os 
import sys
import functools
import string
import datetime
from datetime import date
import snowflake.connector
# Json for the login info
import json
print("----------------------------------------")

----------------------------------------


In [20]:
# This section is to output the Versions of the Data Science Libraries
#
# IMPORTS
print("Check All Versions of the Data Science Libraries we need:")
print("----------------------------------------")
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy
print('numpy: {}'.format(numpy.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas
print('pandas: {}'.format(pandas.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))
print('Snowflake Python Connector: {}'.format(snowflake.connector.__version__))
print("----------------------------------------")

Check All Versions of the Data Science Libraries we need:
----------------------------------------
Python: 3.8.12 (default, Oct 12 2021, 06:23:56) 
[Clang 10.0.0 ]
scipy: 1.7.1
numpy: 1.20.3
matplotlib: 3.4.3
pandas: 1.3.4
sklearn: 0.24.2
Snowflake Python Connector: 2.7.4
----------------------------------------


In [21]:
# ACCESS JSON FILE
FullPath = '/Users/ecrean/scripts/' + 'Sandbox.json'
print("Open and read the JSON File with the connection info: " + FullPath)
if not os.path.exists(FullPath):
   print("JSON File with Source Connection Information not found...")
   sys.exit(1)
with open(FullPath, 'r') as JSONOpen:
   JSONSrcObj = json.load(JSONOpen)

print("Set Connection Variables from JSON")
# This is being done manually here....
# We could also turn this into a method call in the SnowCat class and test that
print(JSONSrcObj['CONNECTION']['REGION'])
REGION = JSONSrcObj['CONNECTION']['REGION']
print(JSONSrcObj['CONNECTION']['ACCT'])
ACCT =   JSONSrcObj['CONNECTION']['ACCT']
print(JSONSrcObj['CONNECTION']['USER'])
USER =   JSONSrcObj['CONNECTION']['USER']
PASS =   JSONSrcObj['CONNECTION']['PASS']
print(JSONSrcObj['CONNECTION']['DW'])
DW =     JSONSrcObj['CONNECTION']['DW']
print(JSONSrcObj['CONNECTION']['DB'])
DB =     JSONSrcObj['CONNECTION']['DB']
print(JSONSrcObj['CONNECTION']['SCHEMA'])
SCHEMA = JSONSrcObj['CONNECTION']['SCHEMA']
print(JSONSrcObj['CONNECTION']['ROLE'])
ROLE =   JSONSrcObj['CONNECTION']['ROLE']
print("----------------------------------------")

Open and read the JSON File with the connection info: /Users/ecrean/scripts/Sandbox.json
Set Connection Variables from JSON
us-west-2
aws_cas2
ECREAN
DEMO_WH
CREAN_DB
SANDBOX
SYSADMIN
----------------------------------------


In [22]:
# Test creating a Python Connector Connection (Works)
# Now make a Basic Connection with the Python Connector
#
import snowflake.connector

# Connect to the DW and execute a simple test query
print("Open Connection: " + REGION + ":" + ACCT + ":" + USER )
conn = snowflake.connector.connect(
       region=REGION,
       account=ACCT,
       user=USER,
       password=PASS,
       warehouse=DW,
       database=DB,
       schema=SCHEMA,
       role=ROLE,
       )

#SQL = "SELECT CURRENT_USER();"
print("Get Current Version: ")
SQL = "SELECT current_version();"

curr = conn.cursor()
curr.execute(SQL);

print("QueryID: " + curr.sfqid )
resultset = curr.fetchall()
for row in resultset:
    output = str(row)
    print(output)

print("----------------------------------------")

Open Connection: us-west-2:aws_cas2:ECREAN
Get Current Version: 
QueryID: 01a2a3ff-0403-b298-0000-432918b7dd2e
('6.4.1',)
----------------------------------------


In [23]:
# Import and use the SnowPyContrib Class and Functions
#
# SnowPytContrib.py would be downloaded and placed in your run directory
# 
# Instantiate the Class as Source, Set the Login Parameters, Create a Connectioin
# Once that is done, you can use any of the methods.
#
import sys
sys.path.insert(0,'/Users/ecrean/scripts/')
from SnowPyContrib import SnowPyContrib
Source = SnowPyContrib()
Source.SetConnVariables(REGION, ACCT, USER, PASS, ROLE, DW, DB, SCHEMA)
Source.GetConn()
# Set some SQL statement in the class
SQLString = 'SELECT current_version();'
Source.SQL = 'SELECT current_version();'
# Execute that SQL
Source.Execute()
# Execute that SQL and display the results
print("Version of Snowflake:")
Source.ExecuteDisplay()
print("Use the pass the SQL method for running SQL and displaying")
Source.ExecuteSQLDisplay(SQLString)

print("----------------------------------------")


Attempt to Open Connection: us-west-2:aws_cas2:ECREAN:DEMO_WH:CREAN_DB
Got the Connection
Get a Cursor
Got a Cursor
Execute: SELECT current_version();
Version of Snowflake:
Execute: SELECT current_version();
('6.4.1',)
Use the pass the SQL method for running SQL and displaying
Execute: SELECT current_version();
('6.4.1',)
----------------------------------------


In [9]:
# Run this SQL in a snowflake session (Could be executed within this notebook, if we want to change that)
#
# use ROLE sysadmin;
# use warehouse demo_wh;
# use database CREAN_DB;
#
# CREATE SCHEMA IF NOT EXISTS SANDBOX;
# use schema SANDBOX;
#
# -----------------------------------
# -- EXAMPLE WITH DOUBLE
# -----------------------------------
# -- Create Test Table
# -- You have to use DOUBLE or something that will preserve the decimal places
# -- If you don't, the math will be all wrong and your results wrong...
# -- Use DOUBLE if you can so that your math is more accurate and you process with more decimal places
# CREATE OR REPLACE TABLE SANDBOX.HOUSE_DATA_POC (
#    PRICE DOUBLE,
#    SQRFT DOUBLE,
#    ACRES DOUBLE,
#    BATH DOUBLE
# );
# 
# -- Insert test data
# INSERT INTO SANDBOX.HOUSE_DATA_POC VALUES (179000.0, 3060.0, 0.75, 2.0);
# INSERT INTO SANDBOX.HOUSE_DATA_POC VALUES (126500.0, 1600.0, 0.26, 1.5);
# INSERT INTO SANDBOX.HOUSE_DATA_POC VALUES (134500.0, 2000.0, 0.7,  1.0);
# INSERT INTO SANDBOX.HOUSE_DATA_POC VALUES (125000.0, 1300.0, 0.65, 1.0);
# 
# SELECT * FROM SANDBOX.HOUSE_DATA_POC;
#
print("Pass SQL to source.SQL and execute to show the rows from the HOUSE_DATA_POC table")
SQL = 'SELECT * FROM SANDBOX.HOUSE_DATA_POC ;'
Source.SQL = SQL
Source.ExecuteDisplay()
print("----------------------------------------------")


Pass SQL to source.SQL and execute to show the rows from the HOUSE_DATA_POC table
Execute: SELECT * FROM SANDBOX.HOUSE_DATA_POC ;
(179000.0, 3060.0, 0.75, 2.0)
(126500.0, 1600.0, 0.26, 1.5)
(134500.0, 2000.0, 0.7, 1.0)
(125000.0, 1300.0, 0.65, 1.0)
----------------------------------------------


In [10]:
# Create another table with strings in it
#
# CREATE OR REPLACE TABLE SANDBOX.HOT_ENCODE_POC (
#   PRICE DOUBLE,
#   SQRFT DOUBLE,
#   ACRES DOUBLE,
#   BATH DOUBLE,
#   BOOLSTR BOOLEAN,
#   STATESTR VARCHAR,
#   JUNKSTR  VARCHAR
#);
#
# -- Insert test data
# INSERT INTO SANDBOX.HOT_ENCODE_POC VALUES (179000.0, 3060.0, 0.75, 2.0, TRUE, 'HOT', 'RED'   );
# INSERT INTO SANDBOX.HOT_ENCODE_POC VALUES (126500.0, 1600.0, 0.26, 1.5, TRUE, 'COLD', 'BLUE'  );
# INSERT INTO SANDBOX.HOT_ENCODE_POC VALUES (134500.0, 2000.0, 0.7,  1.0, FALSE, 'WARM', 'GREEN' );
# INSERT INTO SANDBOX.HOT_ENCODE_POC VALUES (125000.0, 1300.0, 0.65, 1.0, TRUE, 'PLASMA','RED' );
# INSERT INTO SANDBOX.HOT_ENCODE_POC VALUES (150000.0, 1300.0, 0.65, 1.0, TRUE, 'COLD', 'BLUE'  );
# INSERT INTO SANDBOX.HOT_ENCODE_POC VALUES (160000.0, 1300.0, 0.65, 1.0, TRUE, 'COLD', 'GREEN'  );
#
# SELECT * FROM SANDBOX.HOT_ENCODE_POC;
#

print("Pass SQL to source.SQL SELECT the rows from the HOT_ENCODE_POC table showing the initial state.")
SQL = 'SELECT * FROM SANDBOX.HOT_ENCODE_POC ;'
Source.SQL = SQL
Source.ExecuteDisplay()
print("----------------------------------------------")


Pass SQL to source.SQL SELECT the rows from the HOT_ENCODE_POC table showing the initial state.
Execute: SELECT * FROM SANDBOX.HOT_ENCODE_POC ;
(179000.0, 3060.0, 0.75, 2.0, True, 'HOT', 'RED')
(126500.0, 1600.0, 0.26, 1.5, True, 'COLD', 'BLUE')
(134500.0, 2000.0, 0.7, 1.0, False, 'WARM', 'GREEN')
(125000.0, 1300.0, 0.65, 1.0, True, 'PLASMA', 'RED')
(150000.0, 1300.0, 0.65, 1.0, True, 'COLD', 'BLUE')
(160000.0, 1300.0, 0.65, 1.0, True, 'COLD', 'GREEN')
----------------------------------------------


In [11]:
print("Pass SQL to source.SQL to get a table count and return the number of rows found")
SQL = 'SELECT COUNT(*) FROM HOT_ENCODE_POC;'
Source.SQL = SQL
Source.ExecuteDisplay()
print("----------------------------------------------")

print("Pass SQL to source.SQL to get a table count but get a metadata list first and a description from the cursor before executing")
SQL = 'SELECT COUNT(*) FROM HOT_ENCODE_POC;'
Source.SQL = SQL
Source.ExecuteDisplayDescriptionTest()
print("----------------------------------------------")

HouseTable = 'HOUSE_DATA_POC'
InTable = 'HOT_ENCODE_POC'
OutTable = 'HOT_ENCODE_POC_TRAN'

print("Get Table Count using ExecuteGetTableCount() and passing the table name")
TableCount = Source.ExecuteGetTableCount(InTable)
print(TableCount)
print("----------------------------------------------")

Pass SQL to source.SQL to get a table count and return the number of rows found
Execute: SELECT COUNT(*) FROM HOT_ENCODE_POC;
(6,)
----------------------------------------------
Pass SQL to source.SQL to get a table count but get a metadata list first and a description from the cursor before executing
Describe SQL before running it
COUNT(*)
Execute: SELECT COUNT(*) FROM HOT_ENCODE_POC;
Snowflake QueryID:
01a2a3f8-0403-b298-0000-432918b7d7ea
COUNT(*)
(6,)
----------------------------------------------
Get Table Count using ExecuteGetTableCount() and passing the table name
Execute: SELECT COUNT(*) FROM HOT_ENCODE_POC;
6
6
----------------------------------------------


In [13]:
# Now run the One Hot Encoding table function
# Then run the Corellation some Xs function
# Then run the Corellation all Xs function
# Then run the all Correlation function
#
HouseTable = 'HOUSE_DATA_POC'
InTable = 'HOT_ENCODE_POC'
OutTable = 'HOT_ENCODE_POC_TRAN'

print()
print("Now Testing Data Science Functions (Python Functions)")
print()
print("Call SciOneHotEncode_TF:")
Source.SciOneHotEncode_TF(InTable, OutTable)
print("----------------------------------------------")

print("Select from that One Hot Encoded Table to Verify it worked:")
Source.SQL = "SELECT * FROM HOT_ENCODE_POC_TRAN;"
Source.ExecuteDisplay()
print("----------------------------------------------")

print("----------------------------------------------")



Now Testing Data Science Functions (Python Functions)

Call SciOneHotEncode_TF:
Execute: SELECT a.ORDINAL_POSITION, a.COLUMN_NAME, a.DATA_TYPE FROM CREAN_DB.information_schema.columns a WHERE TABLE_SCHEMA = 'SANDBOX' AND TABLE_NAME = 'HOT_ENCODE_POC' ORDER BY a.ORDINAL_POSITION;
(1, 'PRICE', 'FLOAT')
(2, 'SQRFT', 'FLOAT')
(3, 'ACRES', 'FLOAT')
(4, 'BATH', 'FLOAT')
(5, 'BOOLSTR', 'BOOLEAN')
(6, 'STATESTR', 'TEXT')
(7, 'JUNKSTR', 'TEXT')
Debug: select from information schema worked
Debug: start loop
In Loop
(1, 'PRICE', 'FLOAT')
Debug: set ColName
Debug: set DataType
Debug: Entering if logic
In Loop
(2, 'SQRFT', 'FLOAT')
Debug: set ColName
Debug: set DataType
Debug: Entering if logic
In Loop
(3, 'ACRES', 'FLOAT')
Debug: set ColName
Debug: set DataType
Debug: Entering if logic
In Loop
(4, 'BATH', 'FLOAT')
Debug: set ColName
Debug: set DataType
Debug: Entering if logic
In Loop
(5, 'BOOLSTR', 'BOOLEAN')
Debug: set ColName
Debug: set DataType
Debug: Entering if logic
In Loop
(6, 'STATESTR',

In [14]:
print("Select from that One Hot Encoded Table to Verify it worked:")
Source.SQL = "SELECT * FROM HOUSE_DATA_POC;"
Source.ExecuteDisplay()

print("Call Correlation but pass Y and multiple Xs")
Source.SciCorrTablePassY_PassXs(HouseTable, 'Price', 'SQRFT,ACRES,BATH')
print("----------------------------------------------")

print("----------------------------------------------")


Select from that One Hot Encoded Table to Verify it worked:
Execute: SELECT * FROM HOUSE_DATA_POC;
(179000.0, 3060.0, 0.75, 2.0)
(126500.0, 1600.0, 0.26, 1.5)
(134500.0, 2000.0, 0.7, 1.0)
(125000.0, 1300.0, 0.65, 1.0)
Call Correlation but pass Y and multiple Xs
----------------------------------------
SQRFT,ACRES,BATH
(0.973715862872736, 0.5366768095446238, 0.8291101266643618)
----------------------------------------------
----------------------------------------------


In [16]:

print("Select from that One Hot Encoded Table to Verify it worked:")
Source.SQL = "SELECT * FROM HOT_ENCODE_POC_TRAN;"
Source.ExecuteDisplay()
print("----------------------------------------------")
   
print("Call Correlation but pass Y and every other numeric column is an X")
Source.SciCorrTablePassY_AllXs(OutTable, 'Price')
print("----------------------------------------------")



Select from that One Hot Encoded Table to Verify it worked:
Execute: SELECT * FROM HOT_ENCODE_POC_TRAN;
(179000.0, 3060.0, 0.75, 2.0, 1, 1, 0, 0, 0, 1, 0, 0)
(126500.0, 1600.0, 0.26, 1.5, 1, 0, 1, 0, 0, 0, 1, 0)
(134500.0, 2000.0, 0.7, 1.0, 0, 0, 0, 1, 0, 0, 0, 1)
(125000.0, 1300.0, 0.65, 1.0, 1, 0, 0, 0, 1, 1, 0, 0)
(150000.0, 1300.0, 0.65, 1.0, 1, 0, 1, 0, 0, 0, 1, 0)
(160000.0, 1300.0, 0.65, 1.0, 1, 0, 1, 0, 0, 0, 0, 1)
----------------------------------------------
Call Correlation but pass Y and every other numeric column is an X
Execute: SELECT a.ORDINAL_POSITION, a.COLUMN_NAME, a.DATA_TYPE FROM CREAN_DB.information_schema.columns a WHERE TABLE_SCHEMA = 'SANDBOX' AND TABLE_NAME = 'HOT_ENCODE_POC_TRAN' ORDER BY a.ORDINAL_POSITION;
(1, 'PRICE', 'FLOAT')
(2, 'SQRFT', 'FLOAT')
(3, 'ACRES', 'FLOAT')
(4, 'BATH', 'FLOAT')
(5, 'BOOLSTR', 'NUMBER')
(6, 'STATESTR_HOT', 'NUMBER')
(7, 'STATESTR_COLD', 'NUMBER')
(8, 'STATESTR_WARM', 'NUMBER')
(9, 'STATESTR_PLASMA', 'NUMBER')
(10, 'JUNKSTR_RED

In [17]:
print("Call Correlation but every column is an X")
Source.SciCorrTableAllXs(OutTable)
print("----------------------------------------------")

print("----------------------------------------------")


Call Correlation but every column is an X
Execute: SELECT a.ORDINAL_POSITION, a.COLUMN_NAME, a.DATA_TYPE FROM CREAN_DB.information_schema.columns a WHERE TABLE_SCHEMA = 'SANDBOX' AND TABLE_NAME = 'HOT_ENCODE_POC_TRAN' ORDER BY a.ORDINAL_POSITION;
(1, 'PRICE', 'FLOAT')
(2, 'SQRFT', 'FLOAT')
(3, 'ACRES', 'FLOAT')
(4, 'BATH', 'FLOAT')
(5, 'BOOLSTR', 'NUMBER')
(6, 'STATESTR_HOT', 'NUMBER')
(7, 'STATESTR_COLD', 'NUMBER')
(8, 'STATESTR_WARM', 'NUMBER')
(9, 'STATESTR_PLASMA', 'NUMBER')
(10, 'JUNKSTR_RED', 'NUMBER')
(11, 'JUNKSTR_BLUE', 'NUMBER')
(12, 'JUNKSTR_GREEN', 'NUMBER')
----------------------------------------
        PRICE,          SQRFT,          ACRES,          BATH,          BOOLSTR,          STATESTR_HOT,          STATESTR_COLD,          STATESTR_WARM,          STATESTR_PLASMA,          JUNKSTR_RED,          JUNKSTR_BLUE,          JUNKSTR_GREEN,          
('PRICE - 1', 0.6058803648729615, 0.5504786240835118, 0.5291207210628006, 0.26148049946107765, 0.7652149910699156, -0.01719670